In [237]:

import gzip, shutil
import os 
import data_utils as du 
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
from six.moves import cPickle as pickle
import numpy as np
import os
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt 
import platform
import data_utils as du 
from cnn_model import *
from solver import Solver
%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload
plt.rcParams['figure.figsize'] = (15.0, 15.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

Data_folder = 'CIFAR10_Data'
Data_fn = 'cifar-10-python'
X_train, y_train, X_val, y_val, X_test, y_test = du.get_CIFAR10_data(subtract_mean =True)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int64
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [238]:
import qunatize_resnet20 as q_res
import quantize_layers as ql
data_set = {
    'train_data': X_train,
    'train_label': y_train,
    'val_data':X_val,
    'val_label':y_val,
}
train_options = {
    'optimizer': 'adam',
    'lr': 0.001,
    'epoch_num': 20,
    'from_logits':True,
    'save_dir': 'resnet20_model'
}
qunatizer = ql.ModelQunatize() 
q_res20_model = q_res.Resnet20(quantizer=qunatizer)

solver = Solver(q_res20_model,data_set,train_options,save_model=True)
#solver.train()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [239]:
solver.test_model(X_test,y_test)

Loss: 2022.33154296875,Test Accuracy: 9.949999809265137


(<tf.Tensor: shape=(), dtype=float32, numpy=2022.3315>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0995>)

## Show layers and quantizable layers 

In [240]:
import tensorflow_hub as hub 
import quantize_util as qu 
#prepare quntize model model_q
qunatizer = ql.ModelQunatize()
model_path = 'resnet20_model/trained_15'
full_model = hub.KerasLayer(model_path,trainable=True)
model_q = q_res.Resnet20(qunatizer)
qu.copy_weight(model_q,full_model)

train_options = {
    'optimizer': 'adam',
    'lr': 0.001,
    'epoch_num': 20,
    'from_logits':True,
    'save_dir': 'resnet20_model'
}
solver = Solver(model_q,data_set,train_options,save_model=True)




In [241]:
solver = Solver(model_q,data_set,train_options,save_model=True)
q_layer_num = model_q.get_quantizable_layer_count() 
wlfl_list = [[12,4]]*20
qunatizer.set_wlfl_list(wlfl_list)
qunatizer.set_full_precision_mode()
print('Accuracy in full precision mode:')
solver.test_model(X_test,y_test)

print('Accuracy in wlfl mode:')
qunatizer.set_quantize_mode()
solver.test_model(X_test,y_test)



Accuracy in full precision mode:
Loss: 0.9363390803337097,Test Accuracy: 69.91000366210938
Accuracy in wlfl mode:
Loss: 0.9290823936462402,Test Accuracy: 69.41000366210938


(<tf.Tensor: shape=(), dtype=float32, numpy=0.9290824>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6941>)

### Test on quantize_var function 

In [242]:
wl = 16
fl = 8
quantizable_layers = model_q.get_quantizable_layers() 
for layer in quantizable_layers:
    cur_layer = layer
    for var in cur_layer.trainable_variables:
        qu.quantize_var(var,wl,fl)


quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting


In [243]:
qunatizer.set_wlfl_list(wlfl_list)
qunatizer.set_full_precision_mode()
print('Accuracy in full precision mode:')
solver.test_model(X_test,y_test)

print('Accuracy in wlfl mode:')
qunatizer.set_quantize_mode()
solver.test_model(X_test,y_test)

Accuracy in full precision mode:
Loss: 0.9395156502723694,Test Accuracy: 69.94000244140625
Accuracy in wlfl mode:
Loss: 0.9318090081214905,Test Accuracy: 69.43000030517578


(<tf.Tensor: shape=(), dtype=float32, numpy=0.931809>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6943>)

In [244]:
var = -125.999
var = qu.to_fixpoint(var,10,2)
print(var)

tf.Tensor(-126.0, shape=(), dtype=float32)


In [245]:
q_layers = model_q.get_quantizable_layers()
for layer in q_layers:
    #print(layer[0].kernel)
    print(layer)
a = np.abs(q_layers[0].kernel.numpy())
#print(a.reshape(-1))
c = a.reshape(-1).shape[0]
print(q_layers[0].kernel)



quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
<tf.Variable 'quantize_conv2d_798/kernel:0' shape=(3, 3, 3, 16) dtype=float32, numpy=
array([[[[-0.0390625 ,  0.03125   ,  0.08203125,  0.4375    ,
          -0.30859375,  0.25      , -0.04296875, -0.3125    ,
           0.01953125, -0.453125  , -0.03125   ,  0.078125  ,
           0.265625  ,  0.0234375 , -0.34765625,  0.22265625],
         [-0.23828125, -0.16015625,  0.01171875, -0.50390625,
           0.40234375, -0.01953125,  0.3984375 ,  0.0078125 ,
          -0.3125    

## Test WLsearch agent 

#### Test get_parameter_size 


In [246]:
import quantize_complier as qc 
q_layers = model_q.get_quantizable_layers()
wl_agent = qc.WLSearchAgent(model_q,q_layers)
total_size = wl_agent.get_parameter_size()
total_size = total_size
print('Total trainable parameter: {}'.format(total_size))
q_layers_copy = q_layers
q_data = q_layers[0].kernel 

#print(q_layers[0].kernel)
print('=============================')
#print(q_data)
print(len(wl_agent.layers_param_list))
print(wl_agent.wl_list.layer(15).kernel_wl)
wl_agent.get_intger_bits(-3)



quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
Total trainable parameter: (268346, [[432, 0], [2304, 0], [2304, 0], [2304, 0], [2304, 0], [2304, 0], [2304, 0], [4608, 0], [9216, 0], [9216, 0], [9216, 0], [9216, 0], [9216, 0], [18432, 0], [36864, 0], [36864, 0], [36864, 0], [36864, 0], [36864, 0], [640, 10]])
20
32


4

### Test init_search and compression_rate

In [247]:
wl_agent.init_search()
print(wl_agent.wl_list)
wl_agent.get_compression_rate()

<quantize_complier.WlflList object at 0x7f4d504386d0>[8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,0,16][8,16,8,16]


(0.25, 2146768)

## Test SerachFL agent 

### Test search_layer_fl

In [248]:
#q_layers = wl_agent.get_quantize_layers()

data_set = {
    'train_data': X_train,
    'train_label': y_train,
    'val_data':X_val,
    'val_label':y_val,
}
train_options = {
    'optimizer': 'adam',
    'lr': 0.001,
    'epoch_num': 20,
    'from_logits':True,
    'save_dir': 'resnet20_model'
}
def acc_fn(model):
    solver = Solver(model_q,data_set,save_model=False)
    solver.test_model(X_test,y_test)
fl_agent = qc.FlSerachAgent(model_q,solver,acc_fn)
print(len(wl_agent.get_wl_list()))
print(len(q_layers))
fl_agent.search_layers_fl(q_layers,wl_agent.get_wl_list())
print(wl_agent.get_wl_list())
fl_agent.apply_wlfl_to_layers(q_layers,wl_agent.get_wl_list())
#fl_agent.model_accuracy()
#solver.test_model(X_test,y_test)
model_q(X_test)

20
20
<quantize_complier.WlflList object at 0x7f4d504386d0>[8,7,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,0,0][8,8,8,7]


<tf.Tensor: shape=(10000, 10), dtype=float32, numpy=
array([[ -1.1140418 ,  -3.0844963 ,  -0.19119889, ...,  -3.4918647 ,
          3.9700978 ,  -3.0854871 ],
       [  3.0226173 ,   7.9003086 ,  -3.0500617 , ..., -10.4363    ,
         10.445696  ,   0.5678308 ],
       [  3.4073093 ,  -1.6375792 ,  -1.2897418 , ...,  -1.7099354 ,
          4.972007  ,  -1.7667176 ],
       ...,
       [ -3.7454298 ,  -5.1136103 ,  10.919554  , ...,   1.1329545 ,
         -3.4651012 ,  -7.9818344 ],
       [  2.680488  ,   8.803678  ,   1.5420569 , ...,  -7.6888943 ,
         -2.824695  ,   2.2716513 ],
       [ -0.6138832 ,  -7.2697687 ,   1.4499471 , ...,  11.590846  ,
         -3.1202664 ,  -6.6720796 ]], dtype=float32)>

quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
quantize layer extracting
20
20
20
